In [22]:
# means that my matplotlib graphs will be included in the notebook, next to the code
%matplotlib inline

import os

import astropy
import random
import numpy as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u

from hetdex_api.config import HDRconfig
from hetdex_api.detections import Detections
from hetdex_api.elixer_widget_cls import ElixerWidget

In [2]:
# not sure why the code below is here, it was in the Detections database and API notebook
# https://github.com/HETDEX/hetdex_api/blob/master/notebooks/api-notebooks/03-Detections_Database_and_API.ipynb

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [26]:
# Opening H20 NEP catalog and converting it into a pandas DF
H20_NEP_catalog = fits.open('H20_NEP_VIRUS_OVERLAP_CAT_10_2021.fits', memmap = True)
H20_NEP_data = H20_NEP_catalog[1].data
H20_NEP_DF = pd.DataFrame(H20_NEP_data, columns=HDR3_data.columns.names)

In [4]:
# Opening HDR3 detections catalog  ** double check this <-- statement ** and converting it into a pandas DF
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
#fits.open(filename, memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

In [35]:
# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# From the original DF, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]
reduced_h20nep_df = H20_NEP_DF.loc[:, useful_hdr3_cols]
print(reduced_hdr3_df.size)
print(reduced_h20nep_df.size)

38267280
16100448


In [36]:
# Removing data from before 2017 because it isn't good (not useful to us)
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

removed_bad_shots_h20nep_df = reduced_h20nep_df[reduced_h20nep_df.shotid.values >= 20180000000]

print(removed_bad_shots_hdr3_df.size)
print(removed_bad_shots_h20nep_df.size)

37544112
0


Is this enough to make a noise sample? Do I need to do anything else with the data? Seems too easy but I don't know.

We want no source. This catalog has HETDEX detections one. 

    Want to make sure:
        1.No hetdection detec
        
        2. No imaging counterpart. Do some cross-matching. Gives us a 0 and THEN we extract. Want to extract in basically empty space. Start with 100. 
        
            Coordinates still. Trying to see if no match with the .fits file.

### Question: Do I need to keep this signal to noise filter?


Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.

Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff

In [31]:
# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_df['sn'] > 6.5]
print(signal_to_noise_interval.size)

9101040


### Question about 'field' column.

Do I need to specify a field? cause it says there are some egs. Not sure if that's in the scope of my project.

For now anywhere, but once trained. NEP Field!

In [8]:
signal_to_noise_interval

,source_id,detectid,selected_det,ra_mean,dec_mean,fwhm,shotid,field,ra,dec,...,sn,sn_err,chi2,chi2_err,linewidth,linewidth_err,plya_classification,z_hetdex,z_hetdex_conf,combined_plae
25155,3010000025254,3000462158,False,7.868332,0.030342,1.309933,20180104007,dex-fall,7.868559,0.029655,...,6.780000,0.44,1.04,0.22,2.77,0.38,0.001,0.403866,0.900000,0.010113
25156,3010000025378,3000462159,True,7.871345,0.028024,1.309933,20180104007,dex-fall,7.871345,0.028024,...,7.310000,0.48,1.00,0.22,2.84,0.41,0.001,0.243058,0.232867,0.244185
25157,3010000025254,3000462160,False,7.868332,0.030342,1.309933,20180104007,dex-fall,7.867933,0.031079,...,6.810000,0.51,0.99,0.22,2.97,0.42,0.001,0.403866,0.900000,0.010610
25162,3010000025373,3000462181,True,7.876181,0.047542,1.309933,20180104007,dex-fall,7.876181,0.047542,...,9.290000,0.51,1.02,0.22,3.29,0.34,0.001,0.377182,0.800000,0.002744
25173,3010000025362,3000462281,True,7.946422,0.046738,1.309933,20180104007,dex-fall,7.946422,0.046738,...,17.230000,0.52,0.98,0.22,2.76,0.15,0.001,0.354563,0.232867,0.001408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365978,3010001385675,3013842807,True,215.014282,52.901100,1.683606,20220404017,egs,215.014282,52.901100,...,20.700001,0.43,1.03,0.22,3.40,0.19,0.001,0.373095,0.800000,0.001000
1365999,3010001385656,3013842935,True,214.925995,52.822315,1.683606,20220404017,egs,214.925995,52.822315,...,7.270000,0.53,1.07,0.22,3.05,0.41,0.001,0.369006,0.980000,0.030379
1366007,3010001385648,3013842979,True,214.790756,52.778305,1.683606,20220404017,egs,214.790756,52.778309,...,11.080000,0.45,1.00,0.23,3.40,0.37,0.001,0.295676,0.980000,0.037425
1366016,3010001385639,3013843028,True,214.908493,52.784447,1.683606,20220404017,egs,214.908493,52.784447,...,12.300000,0.64,1.34,0.22,3.13,0.16,0.001,0.200016,0.800000,0.009683


In [9]:
# picks a random source for us, the 1 means a single random source.
# Might need to find a different way of getting a random source, just because I don't think
# there's a way to control which source this gets, so it'll always be a different source!
# Or just run this once
random_source = signal_to_noise_interval.sample(n = 1)

In [10]:
# We are using an offset to try to pick an area where there is no source.
# This is a value we are experimenting with, going to go with 20 arcseconds for now
offset = 20 * u.arcsec
# need to convert to degrees so I can add to the ra and dec in catalog,
# the ra and dec in catalog are in degrees
offset = offset.to('deg')

In [11]:
# Applying an offset, then we check if there is a source at the offset!

delta_ra = random_source['ra'] + offset # I forgot how units work here, are these all in arcseconds?
delta_dec = random_source['dec'] + offset # I forgot how units work here, are these all in arcseconds?

## Is a offset of 5 arcseconds a lot? It's a pretty small change
Something we can experiment with!

In [12]:
random_source['ra']

817762    223.642654
Name: ra, dtype: float32

In [13]:
delta_ra.values[0]

223.64821

In [14]:
random_source['dec']

817762    53.828892
Name: dec, dtype: float32

In [15]:
delta_dec.values[0]

53.834446

## Should I make this check an or?
Because this is saying, if the ra AND the dec are not in the catalog, then there is no source.

But if only one isn't there then it's possible the source isn't in the catalog either right?

Here I'm checking if the source in the HDR3 detections catalog? Ask about the catalog name because I think I'm getting mixed up.

In [16]:
# This truth_check df will check if the delta_ra and delta_dec are in the catalog. 
# If the size of this df is 0, then there is no source in this catalog with those specific ra and dec.
truth_check_hdr3 = signal_to_noise_interval[(signal_to_noise_interval['ra'] == delta_ra.values[0]) & (signal_to_noise_interval['dec'] == delta_dec.values[0])]

In [19]:
truth_check_hdr3

,source_id,detectid,selected_det,ra_mean,dec_mean,fwhm,shotid,field,ra,dec,...,sn,sn_err,chi2,chi2_err,linewidth,linewidth_err,plya_classification,z_hetdex,z_hetdex_conf,combined_plae


In [23]:
test = astropy.io.fits.open('H20_NEP_VIRUS_OVERLAP_CAT_10_2021.fits')

Start with detection. One approach was fits file with coordinates. 

Or 

Use this but expand upon it. Find RA and DEC of each shot. And randomly extract.

    delta ra and delta dec. Double check if is there a source there. 
    
For noise sample, no need to run through valentina's code. Only focus on High-z after filtering through Valentina's code.

Once we have noise sample.

Run through valentina's code. Hopefully it detects them all as high-z. Cause neither low-z or star.

Two skycoords. Check coordinates to see if HETDEX detection is there. Compare minimum separation. If the difference is smaller than 3 arcseconds. Then there is a source there, so do not extract there.